# ep4-2 stock bot

In this episode, we introduce a bot we developed to trade stocks based on parameterized buy dip-sell rally strategies and real-time buy and signals.  We first define two classes: `market` and `portfolio`.  We use `market` to simulate the opening and closing of the real market and we use `portfolio` to keep track of the stocks we own, the value of the stocks and the trades.  The code block below shows the implementation of the `market` class.

In [1]:
import datetime
from my_stock import read_stock,proc_stock
import pandas as pd
import time
import math

class market:        
    def __init__(self,silent=False):
        self.status = 'unknown'
        self.time = datetime.datetime.today()        
        self.get_status(silent)
        
    def get_status(self,silent=True):
        if silent == False:
            print('query market status...')
        self.time = datetime.datetime.today()
        t_open = self.time.replace(hour=8,minute=30,second=0,microsecond=0)     #   the time of market open
        t_close = self.time.replace(hour=15,minute=0,second=0,microsecond=0)       #   the time of market close
        
        date_today = str(self.time.year)+'-'+str(self.time.month)+'-'+str(self.time.day)        # convert the date in the format of datetime object to a string
        df = read_stock('SPY')      #   query the SPY price from Yahoo finance in order to get the latest date registered in the price log
        df_latest_date = df.iloc[-1]['Date']
        
        if silent == False:
            print('SPY quote is ${:.2f}'.format(df.iloc[-1]['Close']),
                  'last checked at:',
                  '{}-{}-{},{}:{}'.format(self.time.year,self.time.month,self.time.day,self.time.hour,self.time.minute))
        
        if date_today == df_latest_date:            
            if self.time >= t_open and self.time <= t_close:
                self.status = 'open'
            else:
                self.status = 'closed'
        else:            
            self.status = 'closed'
            
        if silent == False:
            print('market',self.status)
        return self.status

It is noted that a default input parameter `silent` was implemented in the `get_status` method of the class.  `get_status` checks the status of the market at a particular time.  The status of the market can be either `open` or `closed`. By default, the `True` value of the `silent` variable regulates that the `get_status` method does not print intermediate messages during the execution.  If a `True` value is given to the `silent` variable, as it is the case when the `market` class is initiated, the `get_status` method will print intermediate progresses.

The code block below shows the implementation of the `portfolio` class.

In [ ]:
class portfolio:
    def __init__(self):
        df_stocks = pd.read_csv('portfolio.csv')
        df_stocks.set_index(['symbol'],inplace=True)
        self.details = df_stocks
        self.n_lines = len(self.details)
        self.total_value = self.details['current value'].sum()
        
    def update(self):  # this method updates the information in the portfolio when it is called
        self.n_lines = len(self.details)
        for i in range(1,self.n_lines):
            symbol = self.details.iloc[i].name
            df = read_stock(symbol)
            # the two lines below update the current prices of stocks in the portfolio
            # and the corresponding total value of each individual stock
            self.details.at[symbol,'current price'] = df.iloc[-1]['Close']
            self.details.at[symbol,'current value'] = df.iloc[-1]['Close'] * self.details.loc[symbol]['number of shares']
        self.total_value = self.details['current value'].sum()  # compute the total value of the portfolio
        
    def buy(self,symbol):
        df = read_stock(symbol)
        purchase_price = df.iloc[-1]['Close']
        n_stock = math.floor(self.details.loc['_CASH']['current value']/purchase_price)
        stock = pd.Series({'purchased on':df.iloc[-1]['Date'],
                 'number of shares':n_stock,'averaged cost':purchase_price,
                 'total cost':n_stock * purchase_price,
                 'current price':purchase_price,'current value':n_stock * purchase_price})
        stock.rename(symbol,inplace=True)        
        print('executing following purchase:')
        print(stock)
        self.details.at['_CASH','current value'] -= n_stock * purchase_price
        self.details = self.details.append(stock)
        self.update()  # this update is quite important as the previous two lines only update the dataframe element in the portfolio
        
        return stock
    
    def sell(self,symbol):
        df = read_stock(symbol)
        sell_price = df.iloc[-1]['Close']
        n_stock = self.details.loc[symbol]['number of shares']
        
        stock = pd.Series({'purchased on':self.details.loc[symbol]['purchased on'],
                 'number of shares':n_stock,'averaged cost':self.details.loc[symbol]['averaged cost'],
                 'total cost':self.details.loc[symbol]['total cost'],
                 'current price':sell_price,'current value':n_stock * sell_price,
                 '% gain': (n_stock * sell_price - self.details.loc[symbol]['total cost'])/
                 self.details.loc[symbol]['total cost']*100})
        stock.rename(symbol,inplace=True)
        print('executing following sell:')
        print(stock)
        self.details.at['_CASH','current value'] += n_stock * sell_price
        self.details.drop([symbol],inplace=True)
        self.update()

The following code block shows the implementation of two functions: `check_buy` and `check_sell`, which monitors the market in real-time and generate the buy and sell signals according to the parameterized buy dip-sell rally algorithms.  Note that `check_buy` and `check_sell` take rows of dataframes as input parameters.  When implemented in this manner, these two functions can later be repeated applied on rows of dataframes via the `.apply(function)` method.

In [2]:
def check_buy(row):
    df = read_stock(row.name)
    df = proc_stock(df,20,row['d'],row['r'])
    if len(df) == 0:
        return False
    else:
        return df.iloc[-1]['Buy']

def check_sell(row,df_SP500):
    df = read_stock(row.name)
    d = df_SP500.loc[row.name]['d']
    r = df_SP500.loc[row.name]['r']
    df = proc_stock(df,20,d,r)
    if len(df) == 0:
        return False
    else:
        if (df.iloc[-1]['Close'] < 0.95 * row['averaged cost']) or df.iloc[-1]['Sell']:
            return True
        else:
            return False

It is noted that a sell signal is generated when the current stock price is on a rally, indicated by a `True` value of `df.iloc[-1]['Sell']` or when the stop-loss threshold of 5% has been reached, indicated by a `True` value of the `df.iloc[-1]['Close'] < 0.95 * row['averaged cost']` logic test.

The code block below show the implementation of the main program.  It is by design that this main program will be executed every day at 9 AM (CDT), which is 30 min. after market open.

In [3]:
market = market()
if market.status == 'closed':
    print('trader bot shuting down...')
else:
    print('trader bot starting up...')
    
    portfolio = portfolio()     #   open the portfolio    
    
    while market.get_status() and (portfolio.n_lines > 1):  # market is open and we own stocks in the portfolio
        # therefore, we need to monitor the market for sell signals
        time.sleep(5*60)   #   frequency of checking the sell signal is set to 5 minutes
        print('checking sell signal...')
        df_SP500 = pd.read_csv('strategy_table.csv')
        df_SP500.set_index(['Symbol'],inplace=True)
        ls_cand = portfolio.details.iloc[1:].apply(check_sell,args=(df_SP500,),axis=1)
        ls_cand = ls_cand[ls_cand==True]  # this list stores the stocks presenting sell signals        
        if (len(ls_cand) == 0):
            print('sell signal not detected')
            portfolio.update()  # update the current value of the portfolio
            print('portfolio value:{:.2f}'.format(portfolio.total_value))
        else:
            for cand in ls_cand.index:
                portfolio.sell(cand)
                
    # the program exits the while loop above under two conditions: market is closed or the portfolio does not contain
    # anymore stocks, under the second condition, we will wait until close to market close to scan the market for 
    # buy signals
                
    if market.get_status():
        t_check_buy = datetime.datetime.today().replace(hour=15,minute=0,second=0,microsecond=0)
        time.sleep((t_check_buy - datetime.datetime.today()).seconds)
        print('start to check buy signal...')
        df_SP500 = pd.read_csv('strategy_table.csv')
        df_SP500.set_index(['Symbol'],inplace=True)
        check_buy_start = time.time()
        df_check_buy = df_SP500.apply(check_buy,axis=1)
        check_buy_end = time.time()
        print('checking buy signal took:',check_buy_end - check_buy_start,'seconds')
        cand_name = df_SP500[df_check_buy].sort_values(by=['Signal Avg. Gain %'],ascending=False).iloc[0].name
        cand_prospect = df_SP500.loc[cand_name]['Signal Avg. Gain %']
        print('symbol:',cand_name,'prospect: {:.2f}%'.format(cand_prospect))
        portfolio.buy(cand_name)
        
    portfolio.details.to_csv('portfolio_'+str(market.time.year)+'-'+
                             str(market.time.month)+'-'+
                             str(market.time.day)+'.csv')
    portfolio.details.to_csv('portfolio.csv')  
    # the two lines above saves the portfolio, two versions are saved, one named after today's date,
    # which will serve as a log, a second named without today's date, which will serve as a running log and 
    # will be loaded on the next trading day.
    print('trader bot shutting down...')

query market status...
SPY quote is $362.23 last checked at: 2020-11-25,11:38
market open
trader bot starting up...


NameError: name 'portfolio' is not defined

It is noted that in the code block above, we implemented an apply method that uses a function with additional positional arguments, i.e., `.apply(check_sell,args=(df_SP500,),axis=1)`.  In general, the syntax for this type of implementation ois as follows: `pandas.DataFrame.apply(function,args=(x1,*))`.  In this implementation, we only need to enter one positional argument in addition to the original element of the series, on which the `.apply` method operates.  Therefore, the second attribute of the tuple was left blank.